In [6]:
from src.flight_analysis.scrape import Scrape

In [7]:
flights = Scrape("CTA", "PMO", "2023-10-28")
flights.run_scrape()

In [8]:
flights.data

,departure_datetime,arrival_datetime,airlines,travel_time,origin,destination,layover_n,layover_time,layover_location,price_eur,price_trend,price_value,access_date,one_way,has_train
0,2023-10-28 07:50:00,2023-10-28 11:25:00,[easyJet],0 days 03:35:00,CTA,PMO,1,0 days 01:30:00,[NAP],58,typical,None,2023-08-07 20:18:11.150904,True,False
1,2023-10-28 14:00:00,2023-10-28 18:20:00,[ITA],0 days 04:20:00,CTA,PMO,1,0 days 01:50:00,[FCO],222,typical,None,2023-08-07 20:18:11.150920,True,False
2,2023-10-28 19:10:00,2023-10-28 22:55:00,[ITA],0 days 03:45:00,CTA,PMO,1,0 days 01:10:00,[FCO],222,typical,None,2023-08-07 20:18:11.150922,True,False
3,2023-10-28 12:50:00,2023-10-28 22:55:00,[ITA],0 days 10:05:00,CTA,PMO,3,NaT,"[LIN, NAP, FCO]",191,typical,None,2023-08-07 20:18:11.150924,True,False
4,2023-10-28 15:25:00,2023-10-28 22:10:00,[ITA],0 days 06:45:00,CTA,PMO,1,0 days 04:20:00,[FCO],222,typical,None,2023-08-07 20:18:11.150926,True,False
5,2023-10-28 11:05:00,2023-10-28 16:20:00,[ITA],0 days 05:15:00,CTA,PMO,1,0 days 01:50:00,[LIN],228,typical,None,2023-08-07 20:18:11.150927,True,False
6,2023-10-28 06:00:00,2023-10-28 09:25:00,[ITA],0 days 03:25:00,CTA,PMO,1,0 days 00:55:00,[FCO],254,typical,None,2023-08-07 20:18:11.150929,True,False
7,2023-10-28 10:15:00,2023-10-28 14:25:00,[ITA],0 days 04:10:00,CTA,PMO,1,0 days 01:35:00,[FCO],265,typical,None,2023-08-07 20:18:11.150931,True,False
8,2023-10-28 07:00:00,2023-10-28 11:55:00,[ITA],0 days 04:55:00,CTA,PMO,1,0 days 02:20:00,[FCO],280,typical,None,2023-08-07 20:18:11.150933,True,False
9,2023-10-28 06:00:00,2023-10-28 11:05:00,[Lufthansa],0 days 05:05:00,CTA,PMO,1,0 days 01:00:00,[MUC],408,typical,None,2023-08-07 20:18:11.150934,True,False


In [9]:
flights._url

'https://www.google.com/travel/flights?q=Flights%20to%20PMO%20from%20CTA%20on%202023-10-28%20oneway&curr=EUR&gl=IT'

In [10]:
raise NotImplementedError()

NotImplementedError: 

In [ ]:
import pandas as pd
from src.flight_analysis.database import Database
import private.private as private

db = Database(
    db_host=private.DB_HOST,
    db_name=private.DB_NAME,
    db_user=private.DB_USER,
    db_pw=private.DB_PW,
)

query = "SELECT uuid, layover_location FROM scraped"

cur = db.conn.cursor()
cur.execute(query)

res = cur.fetchall()
res

In [ ]:
df = pd.DataFrame(res, columns=["flight_uuid", "layover_location"])
df

In [ ]:
# get rid of "min"
df.lauover_location = df.layover_location.str.strip()
df.layover_location = df.layover_location.replace("", np.nan)
df.layover_location = df.layover_location.replace("min", np.nan)
df.layover_location = df.layover_location.replace("Change of airport", np.nan)

df = df.dropna()
df

In [ ]:
def split_dataframe(df, chunk_size=5000):
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i * chunk_size : (i + 1) * chunk_size])
    return chunks


chunks = split_dataframe(df, chunk_size=5000)

i = 1
for c in chunks:
    print(i, "/", len(chunks))
    db.add_pandas_df_to_db(c, table_name=db.table_scraped_layovers)
    i += 1

In [ ]:
# this process is to retroactively split the flight airlines into a separate "scraped_airlines" table,
# in order to violate the 1NF and 2NF rules of database normalization.

raise NotImplementedError("This script is not meant to be run again!!!!!")

import pandas as pd
from src.flight_analysis.database import Database
import private.private as private

# db = Database(
#     db_host=private.DB_HOST,
#     db_name=private.DB_NAME,
#     db_user=private.DB_USER,
#     db_pw=private.DB_PW,
# )

query = "SELECT id, airlines FROM scraped"

cur = db.conn.cursor()
cur.execute(query)

res = cur.fetchall()

df = pd.DataFrame(res, columns=["flight_uuid", "airline"])

df2 = df.explode("airline")
df2["airline"] = df2["airline"].map(lambda x: x.lstrip("'").rstrip("'"))
df2["airline"] = df2["airline"].replace(
    {"Separate tickets booked together": "multiple"}
)
df2 = df2.reset_index(drop=True)


def split_dataframe(df, chunk_size=5000):
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i * chunk_size : (i + 1) * chunk_size])
    return chunks


chunks = split_dataframe(df2, chunk_size=5000)

i = 1
for c in chunks:
    print(i, "/", len(chunks))
    db.add_pandas_df_to_db(c, table_name=db.table_scraped_airlines)
    i += 1

In [ ]:
import re
import numpy as np


def _is_arg_layover(arg):
    """ """
    # layover location cases
    layover_location_cases = {
        # case 1: (xx hr xx min AAA), (xx hr xx min Aaaaa)
        1: re.search("\d{0,2} hr \d{0,2} min [A-Z]+", arg),
        # case 2: (xx hr AAA), (xx hr AAA)
        2: re.search("\d{0,2} hr [A-Z]+", arg),
        # case 3: (xx min AAA), (xx min AAA)
        3: re.search("\d{0,2} min [A-Z]+", arg),
        # case 4: (AAA, BBB, ...)
        4: re.search("^[A-Z]{3}, ([A-Z]{3}(, )?)?", arg),
    }

    print(layover_location_cases)

    return any(layover_location_cases.values())


def _parse_layover_times_location(arg):
    layover_time = None
    layover_location = None

    # layover time
    if (" hr" in arg) or (" min" in arg):
        layover_time = (
            re.search("^(\d{1,2} hr){0,1}\s{0,1}(\d{1,2} min){0,1}\s", arg)
            .group()
            .strip()
        )
        layover_location = arg.split(" ")[-1]

    # layover location
    if "," in arg:
        layover_location = arg.split(", ")
        layover_location = [x.strip() for x in layover_location]

    return layover_time, layover_location


test_args = [
    "22 hr 12 min FCO",
    # "22 hr 12 min",
    # "3 min Ancona",
    # "13 min LAX",
    # "FCO, JFK",
    # "FCO, JFK",
    # "21 hr 5 min DOH",
    # "hehe",
    "SWISS, Singapore AirlinesOperated by Helvetic",
]

for arg in test_args:
    if _is_arg_layover(arg):
        x = _parse_layover_times_location(arg)
        print(arg, x)

In [ ]:
import re

not bool(re.search("ITA, ", "ITA, Singapore"))